In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(device)

cuda


In [2]:
%load_ext autoreload 
%autoreload 2

In [3]:
seed = 42
import os
os.environ['PYTHONHASHSEED'] = str(seed)
import random
random.seed(seed)
import numpy as np
np.random.seed(seed)
import torch
torch.manual_seed(seed)
from envs.environment_t import Environment 
from utils.helper_fcts import display_state, class_to_dict
import torch
from IPython.display import display
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.loggers import CSVLogger
from torch import Tensor, nn
from torch.optim import Adam, Optimizer
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
import random
import pandas as pd
from copy import deepcopy
from itertools import compress, chain
from collections import namedtuple, deque
from itertools import count
import time 
from IPython.display import clear_output
import matplotlib.pyplot as plt
import math
from agent_policy.policy_base_class import *
from agent_policy.policy_cfg import DQNCfg
from utils.dataloader import GridDataLoader
from tqdm import tqdm

ModuleNotFoundError: No module named 'agent_policy.policy_cfg'

In [4]:
### TRAINING CONFIG ###
wandb_run_name = 'thien_dev'
cfg=DQNCfg
cfg_dict=class_to_dict(cfg)
# cfg_dict['training']['BATCH_SIZE']

BATCH_SIZE = cfg_dict['training']['BATCH_SIZE']
GAMMA = cfg_dict['training']['GAMMA']
EPS_START = cfg_dict['training']['EPS_START']
EPS_END = cfg_dict['training']['EPS_END']
EPS_DECAY = cfg_dict['training']['EPS_DECAY']
TAU = cfg_dict['training']['TAU']
LR = cfg_dict['training']['LR']
num_episodes = cfg_dict['training']['NUM_EPISODES']


In [5]:
### Init environment & agent ####
mode = 'training'
data_dir = os.path.join(os.path.abspath(""), "data")
variant = 0  
env = Environment(variant, data_dir)
dl = GridDataLoader(variant, mode, 20000)
# Get number of actions from gym action space
n_actions = 5
# Get the number of state observations
state = env.reset(mode)
n_obs = len(state)
print(n_obs)
num_layers = 10
layers_dims = [128 for i in range(num_layers)]

dqn_agent = DQN_Agent(num_inputs=n_obs, num_outputs=n_actions, layer_dims=layers_dims, activation='lrelu', device=device)

# optimizer = Adam(dqn_agent.policy.parameters(), lr=LR, amsgrad=True)
# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)

8


#### Weight and biases setup

In [6]:
import wandb
wandb.init(
    # set the wandb project where this run will be logged
    project="Warehouse Gridworld",
    # track hyperparameters and run metadata
    config=cfg_dict['training'],
    name=wandb_run_name
)

wandb: Currently logged in as: benoitauclair30 (itsonlygrid). Use `wandb login --relogin` to force relogin


## Traning loop   

In [7]:
# Transition = namedtuple('Transition',('state','action','next_state','reward'))

# def optimize_model(): 
#     if len(dl.memory) < BATCH_SIZE: 
#         return 
    
#     # turn the transition to batch 
#     transitions = dl.sample(BATCH_SIZE)
#     # print(f" Sample Transition {transitions[15]}")
#     batch = Transition(*zip(*transitions))
#     # print(f" Batch for training {batch.action}")
#     non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
#                                           batch.next_state)), device=device, dtype=torch.bool)
#     non_final_next_states = torch.cat([s for s in batch.next_state
#                                                 if s is not None])
#     state_batch = torch.cat(batch.state)
#     action_batch = torch.cat(batch.action)
#     reward_batch = torch.cat(batch.reward)
    
#     # Compute Q(s_t, a)
#     state_action_values = dqn_agent.policy(state_batch).gather(1, action_batch)
#     # --> get the output of the net using the state, where an action was taken
#     # 
    
#     # Comput Vs for all nextstate 
#     next_state_values = torch.zeros(BATCH_SIZE, device=device)
#     with torch.no_grad(): 
#         next_state_values[non_final_mask] = dqn_agent.target(non_final_next_states).max(1)[0]
        
#     expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    
#     # Compute Huber Loss 
#     criterion = nn.MSELoss()
#     loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    
#     # Optimize the model 
#     optimizer.zero_grad()
#     loss.backward()
    
#     # Inplace gradient clipping 
#     # torch.nn.utils.clip_grad_value_(dqn_agent.policy.parameters(), 100)
#     optimizer.step()
    

In [9]:
cummulative_reward = []

print(f'Training Start, running on {device}')

pbar = tqdm(total=num_episodes)

step_done = 0 
for i_expisode in range(num_episodes):
    
    ## Run Validation Reward
    if i_expisode % 100 == 0 : 
        step_done += 0 
        val_rew = 0 
        state = env.reset("validation")
        state = torch.tensor(state, dtype=torch.float32, device = device).unsqueeze(0)
        for j in range(200):  # loop over 200 steps per episode
            act = dqn_agent.select_action(state, greedy=True)  # TODO: get action for the obs from your trained policy
            rew, next_obs, _ = env.step(act)  # take one step in the environment
            val_rew += rew  # track rewards
            obs = next_obs  # continue from the new obs
        wandb.log({
            "val_reward": val_rew
        })    
        # pbar.write(f'Episode {i_expisode} / {num_episodes}, validation reward: {val_rew}')

    # Training LOOP
    state = env.reset(mode)
    state = torch.tensor(state, dtype=torch.float32, device = device).unsqueeze(0)
    
    training_reward  = 0.0
    for t in count():
        action = dqn_agent.select_action(state)
        reward, observation, done = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        training_reward += reward
        if done: 
            next_state = None
        else:             
            next_state = torch.tensor(observation, dtype=torch.float32,device=device).unsqueeze(0)
    
        # store in memory 
        # dl.push(state, action, next_state, reward)
        dqn_agent.save_experience(state, action, next_state, reward)

        state = next_state
        
        training_loss = dqn_agent.optimize()
        
        # Upate target network 
        if step_done % 400 == 0:
            # Soft update of the target network 
            # θ′ ← τ θ + (1 −τ )θ′
            target_net_state_dict = dqn_agent.target.state_dict()
            policy_net_state_dict = dqn_agent.policy.state_dict()
            for key in policy_net_state_dict:
                target_net_state_dict[key] = policy_net_state_dict[key]*TAU \
                    + target_net_state_dict[key]*(1-TAU)
            dqn_agent.target.load_state_dict(target_net_state_dict)
            
        
        if done: 
            wandb.log({
                "training_reward" : training_reward,
                "training_loss": training_loss
            })
            break
    pbar.update(1) 
#     scheduler.step(val_rew)
pbar.close()
print('complete')

Training Start, running on cuda



  0%|                                                 | 0/10000 [00:36<?, ?it/s]


RuntimeError: Index tensor must have the same number of dimensions as input tensor

In [ ]:
pbar.close()
wandb.finish()

### Play the game with the final state

In [ ]:
state = env.reset("validation")
state = torch.tensor(state, dtype=torch.float32, device = device).unsqueeze(0)

for j in range(200):  # loop over 200 steps per episode
    
    act = dqn_agent.select_action(state)  # TODO: get action for the obs from your trained policy
    rew, next_obs, _ = env.step(act)  # take one step in the environment
    # visualization
    clear_output(wait=True)
    print(act)
    display_state(state=env.get_state(), episode="Validation ")
    time.sleep(0.2)
    